## Compare
This simple notebook compares the Pareto-frontiers from two different studies together in one plot.

In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core import ultratb

ultratb.VerboseTB.tb_highlight = "bg:#3e0054"

In [ ]:
try:
    from syftr.configuration import cfg
except:
    import os
    os.chdir('./../')

In [ ]:
from syftr.configuration import cfg

In [ ]:
# TITLE = "Cerebras vs Local Models on FinanceBench (Accuracy vs Latency)"

# STUDY_NAME_1 = "cerebras4--rag-and-agents-local-only--financebench_hf"
# STUDY_NAME_2 = "cerebras4--rag-and-agents-cerebras-only--financebench_hf"

# OBJ2_NAME = "Latency"
# X_UNIT = "Latency (seconds)"
# IS_COST = False

In [ ]:
# TITLE = "Cerebras vs Local Models on PhantomWiki (Accuracy vs Latency)"

# STUDY_NAME_1 = "cerebras4--rag-and-agents-local-only--phantomwikiv050_hf--depth_20_size_10000_seed_3"
# STUDY_NAME_2 = "cerebras4--rag-and-agents-cerebras-only--phantomwikiv050_hf--depth_20_size_10000_seed_3"

# OBJ2_NAME = "Latency"
# X_UNIT = "Latency (seconds)"
# IS_COST = False

In [ ]:
# TITLE = "Cerebras vs Local Models on FinanceBench (Accuracy vs Cost)"

# STUDY_NAME_1 = "cerebras5--rag-and-agents-local-only--financebench_hf"
# STUDY_NAME_2 = "cerebras5--rag-and-agents-cerebras-only--financebench_hf"

# OBJ2_NAME = "Cost"
# X_UNIT = "Cost (Cents per 100 Calls)"
# IS_COST = True

In [ ]:
TITLE = "Cerebras vs Local Models on PhantomWiki (Accuracy vs Cost)"

STUDY_NAME_1 = "cerebras5--rag-and-agents-local-only--phantomwikiv050_hf--depth_20_size_10000_seed_3"
STUDY_NAME_2 = "cerebras5--rag-and-agents-cerebras-only--phantomwikiv050_hf--depth_20_size_10000_seed_3"

OBJ2_NAME = "Cost"
X_UNIT = "Cost (Cents per 100 Calls)"
IS_COST = True

In [ ]:
from syftr.optuna_helper import get_completed_trials


Y_UNIT = "Accuracy (%)"

SUCCESS_RATE = 0.9

STUDY_COLOR_1 = "#5b45f8"
STUDY_COLOR_2 = "#ff5638"

STORAGE = cfg.database.get_optuna_storage()

df_trials_1 = get_completed_trials(STUDY_NAME_1, success_rate=0)
df_trials_2 = get_completed_trials(STUDY_NAME_2, success_rate=0)

n_trials_1 = len(df_trials_1)
n_trials_2 = len(df_trials_2)

STUDY_LABEL_1 = f"Local models ({n_trials_1} trials)"
STUDY_LABEL_2 = f"Cerebras models ({n_trials_2} trials)"

SHOW_ALL_TRIALS = True

if IS_COST:
    df_trials_1["values_1"] *= 10000
    df_trials_2["values_1"] *= 10000

In [ ]:
import pandas as pd
from syftr.optuna_helper import get_pareto_df

df_pareto_1: pd.DataFrame = get_pareto_df(STUDY_NAME_1, SUCCESS_RATE)
df_pareto_1.sort_values("values_0", ascending=True, inplace=True)

df_pareto_2: pd.DataFrame = get_pareto_df(STUDY_NAME_2, SUCCESS_RATE)
df_pareto_2.sort_values("values_0", ascending=True, inplace=True)

if IS_COST:
    df_pareto_1["values_1"] *= 10000
    df_pareto_2["values_1"] *= 10000

In [ ]:
import matplotlib.pyplot as plt

_, ax = plt.subplots(figsize=(12, 6), dpi=300)

if SHOW_ALL_TRIALS:
    ax.scatter(
        df_trials_1["values_1"],
        df_trials_1["values_0"], 
        color=STUDY_COLOR_1,
        zorder=1,
        alpha=0.2,
        s=10,
    )

    ax.scatter(
        df_trials_2["values_1"],
        df_trials_2["values_0"], 
        color=STUDY_COLOR_2,
        zorder=1,
        alpha=0.2,
        s=10,
    )

ax.step(
    df_pareto_1["values_1"],
    df_pareto_1["values_0"], 
    where="post",
    color=STUDY_COLOR_1,
    label=STUDY_LABEL_1,
    zorder=2,
    linewidth=1.5,
    marker="o",
    markersize=5,
)

ax.step(
    df_pareto_2["values_1"],
    df_pareto_2["values_0"], 
    where="post",
    color=STUDY_COLOR_2,
    label=STUDY_LABEL_2,
    zorder=2,
    linewidth=1.5,
    marker="o",
    markersize=5,
)

# ax.set_title(TITLE)
ax.set_xscale("log")
ax.set_xlabel(X_UNIT)
ax.set_ylabel(Y_UNIT)
ax.legend()

In [ ]:
def convert_to_insights(df: pd.DataFrame, title: str = None) -> pd.DataFrame:
    df_insights = df.copy()
    df_insights["Accuracy"] = df_insights["values_0"]
    df_insights[OBJ2_NAME] = df_insights["values_1"]
    df_insights["Title"] = title
    return df_insights

In [ ]:
from kneed import KneeLocator


def set_knee(df_pareto):
    df_knee = df_pareto.sort_values(by="values_1").copy()
    df_knee[["values_0", "values_1"]] = df_knee[["values_0", "values_1"]].astype(float)
    knee = KneeLocator(
        df_knee["values_1"].values,
        df_knee["values_0"].values,
        curve="concave",
        direction="increasing",
    )
    df_pareto["is_knee"] = df_pareto["values_1"] == knee.knee

In [ ]:
from syftr.plotting.insights import generate_trial_description_table


df_pareto_1_insights = convert_to_insights(df_pareto_1)
df_pareto_1_insights = generate_trial_description_table(df_pareto_1_insights)
set_knee(df_pareto_1_insights)
print(df_pareto_1_insights.columns)

df_pareto_2_insights = convert_to_insights(df_pareto_2)
df_pareto_2_insights = generate_trial_description_table(df_pareto_2_insights)
set_knee(df_pareto_2_insights)

df_trials_1_insights = convert_to_insights(df_trials_1)
df_trials_2_insights = convert_to_insights(df_trials_2)

df_pareto_1_insights

In [ ]:
from syftr.plotting.insights import plot_pareto_plot


def plot_with_description(
    df_pareto_desc: pd.DataFrame,
    study_name: str,
    df_pareto_baseline: pd.DataFrame = None,
    is_cost: bool = False,
    df_trials: pd.DataFrame = None,
    df_trials_baseline: pd.DataFrame = None,
    show_baseline_trials=False,
    experiment_color="red",
    experiment_trial_label="Experiment Trials",
    experiment_pareto_label="Pareto-Frontier based on Experiment Trials",
    baseline_color="black",
    baseline_trial_label="Baseline",
    baseline_pareto_label="Baseline Pareto",
    show_title=True,
):
    if SHOW_ALL_TRIALS:
        assert df_trials_baseline is not None, "df_trials_baseline must be provided to show all trials"

    _, ax = plt.subplots(figsize=(14, 10), dpi=300)

    if show_baseline_trials:
        ax.scatter(
            df_trials_baseline["values_1"],
            df_trials_baseline["values_0"], 
            color=baseline_color,
            label=baseline_trial_label,
            zorder=1,
            alpha=0.2,
            s=30,
        )

    if df_pareto_baseline is not None:
        ax.step(
            df_pareto_baseline["values_1"],
            df_pareto_baseline["values_0"], 
            where="post",
            color=baseline_color,
            label=baseline_pareto_label,
            zorder=2,
            linewidth=1.5,
            markersize=5,
        )

    plot_pareto_plot(
        df_pareto_desc=df_pareto_desc,
        study_name=study_name,
        is_cost=is_cost,
        df_trials=df_trials,
        ax=ax,
        show_title=False,
        show_baselines=False,
        show_sota=False,
        trials_label=experiment_trial_label,
        trials_face_color=experiment_color,
        trials_edge_color=experiment_color,
        pareto_label=experiment_pareto_label,
    )

    if show_title:
        ax.set_title(TITLE)
    ax.set_xscale("log")
    ax.set_xlabel(X_UNIT)
    ax.set_ylabel(Y_UNIT)
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), borderaxespad=0.)


In [ ]:
plot_with_description(
    df_pareto_desc=df_pareto_2_insights,
    study_name=STUDY_NAME_2,
    df_pareto_baseline=df_pareto_1_insights,
    is_cost=IS_COST,
    # df_trials=df_trials_2_insights,
    df_trials_baseline=df_trials_1_insights,
    experiment_color=STUDY_COLOR_2,
    experiment_trial_label=STUDY_LABEL_2,
    experiment_pareto_label=f"Pareto-Frontier based on {STUDY_LABEL_2}",
    show_baseline_trials=True,
    baseline_color=STUDY_COLOR_1,
    baseline_trial_label=STUDY_LABEL_1,
    baseline_pareto_label=f"Baseline Pareto-Frontier based on {STUDY_LABEL_1.lower()}",
    show_title=True,
)

In [ ]:
plot_with_description(
    df_pareto_desc=df_pareto_1_insights,
    study_name=STUDY_NAME_1,
    df_pareto_baseline=df_pareto_2_insights,
    is_cost=IS_COST,
    df_trials=df_trials_1_insights,
    df_trials_baseline=df_trials_2_insights,
    experiment_color=STUDY_COLOR_1,
    experiment_trial_label=STUDY_LABEL_1,
    experiment_pareto_label=f"Pareto-Frontier based on {STUDY_LABEL_1.lower()}",
    show_baseline_trials=False,
    baseline_color=STUDY_COLOR_2,
    baseline_trial_label=STUDY_LABEL_2,
    baseline_pareto_label=f"Baseline Pareto-Frontier based on {STUDY_LABEL_2}",
    show_title=True,
)

In [ ]:
print("Study 1")
print("-" * 40)
print(f"Study: {STUDY_LABEL_1}")
print(f"Pareto-Frontier: {len(df_pareto_1)} trials")
print(f"Top-Accuracy Trial: {df_pareto_1_insights.iloc[-1]["Title"]}")
print(f"Knee-Point Trial: {df_pareto_1_insights[df_pareto_1_insights['is_knee']].iloc[0]['Title']}")

In [ ]:
print("Study 2")
print("-" * 40)
print(f"Study: {STUDY_LABEL_2}")
print(f"Pareto-Frontier: {len(df_pareto_2)} trials")
print(f"Top-Accuracy Trial: {df_pareto_2_insights.iloc[-1]["Title"]}")
print(f"Knee-Point Trial: {df_pareto_2_insights[df_pareto_2_insights['is_knee']].iloc[0]['Title']}")